In [1]:
import sqlite3

import pandas as pd
import numpy as np
from urllib.request import urlopen  
import json

getkey = ! cat ../../apikey.txt
apikey = getkey[0]

from Resources.fhelp import get_appended, get_appended2

In [2]:
#get list of tickers (just dow until everything is set up)
dow_url = "https://financialmodelingprep.com/api/v3/dowjones_constituent?apikey=600e501fe324b2f1cbb2aec5b390b6db"
dow_tick = pd.read_json(dow_url)
tickers = list(dow_tick.symbol.values)
print(tickers,)
tickers += ['SECO']

['CRM', 'WBA', 'V', 'NKE', 'UNH', 'TRV', 'VZ', 'INTC', 'WMT', 'JNJ', 'DIS', 'MCD', 'JPM', 'CAT', 'BA', 'AMGN', 'DOW', 'AAPL', 'GS', 'CSCO', 'MSFT', 'HD', 'PG', 'MRK', 'IBM', 'HON', 'KO', 'CVX', 'AXP', 'MMM']


In [3]:
urls = pd.read_json('Resources/urls.json',typ='series')

In [4]:
conn = sqlite3.connect('../../Finances.db')
c = conn.cursor()

In [8]:
tickers = ['AAPL']

In [9]:
schemas = open("schema.sql", "a+")
for name, url in zip(urls.index,urls.values):
    print(name)
    if url =='NONE':
        continue
    tab = get_appended2(url,tickers,apikey,verbose=True)
    tab.drop_duplicates(inplace=True)
    a = pd.io.sql.get_schema(tab,name)
    c.execute('drop table if exists '+ name)
    c.execute(a)
    tosql = a +';\n\n'
    schemas.write(tosql)
    tab.to_sql(name,con=conn,index=False,if_exists='append')


CV_PROFILE


,symbol,price,beta,volAvg,mktCap,lastDiv,range,changes,companyName,currency,...,city,state,zip,dcfDiff,dcf,image,ipoDate,defaultImage,isEtf,isActivelyTrading
0,AAPL,135.37,1.33758,102825663,2272604980000,0.82,53.1525-145.09,0.24,Apple Inc,USD,...,Cupertino,CALIFORNIA,95014,89.92,127.377,https://financialmodelingprep.com/image-stock/...,1980-12-12,False,False,True


CV_QUOTE


,symbol,name,price,changesPercentage,change,dayLow,dayHigh,yearHigh,yearLow,marketCap,...,volume,avgVolume,exchange,open,previousClose,eps,pe,earningsAnnouncement,sharesOutstanding,timestamp
0,AAPL,Apple Inc.,135.37,0.18,0.24,133.6921,135.51,145.09,53.1525,2272604979200,...,60145130,102825663,NASDAQ,134.35,135.13,3.687,36.715485,2021-01-27T16:30:00.000+0000,16788099130,2021-02-14 19:07:53


CV_EXECUTIVES
NAMER-------------


...

TS_DIVIDENDS_HIST
NAMER-------------


,date,label,adjDividend,dividend,recordDate,paymentDate,declarationDate,symbol
0,2021-02-05,"February 05, 21",0.20500,0.205,2021-02-08,2021-02-11,2021-01-27,AAPL
1,2020-11-06,"November 06, 20",0.20500,0.205,2020-11-9,2020-11-12,2020-10-29,AAPL
2,2020-08-07,"August 07, 20",0.20500,0.820,2020-08-10,2020-08-13,2020-07-30,AAPL
3,2020-05-08,"May 08, 20",0.20500,0.820,2020-05-11,2020-05-14,2020-04-30,AAPL
4,2020-02-07,"February 07, 20",0.19250,0.770,2020-02-10,2020-02-13,2020-01-28,AAPL
...,...,...,...,...,...,...,...,...
81,1988-05-16,"May 16, 88",0.00071,NaN,,,,AAPL
82,1988-02-12,"February 12, 88",0.00071,NaN,,,,AAPL
83,1987-11-17,"November 17, 87",0.00071,NaN,,,,AAPL
84,1987-08-10,"August 10, 87",0.00054,NaN,,,,AAPL


TS_SPLITS_HIST
NAMER-------------


,date,label,numerator,denominator,symbol
0,2020-08-31,"August 31, 20",4.0,1.0,AAPL
1,2014-06-09,"June 09, 14",7.0,1.0,AAPL
2,2005-02-28,"February 28, 05",2.0,1.0,AAPL
3,2000-06-21,"June 21, 00",2.0,1.0,AAPL
4,1987-06-16,"June 16, 87",2.0,1.0,AAPL


TS_BIASFREE
TI_DAILY
TI_INTRADAY


In [ ]:
u = urls['TS_PRICE_DAY']
uu = u.replace(' ','AAPL') + apikey
output = json.load(urlopen(uu))
norm = pd.read_json(uu)
display(norm)
norm.shape

In [ ]:
ttdf =  pd.DataFrame()
for i in range(len(norm)):
    ttdf = ttdf.append(pd.Series(norm.iloc[i,1]),ignore_index=True)
    

In [23]:
norm.loc[:,'Series'] = pd.Series(norm.iloc[:,1])

In [33]:
ser = pd.Series(norm.iloc[:,1])

In [46]:
u = urls['TS_PRICE_DAY']
uu = u.replace(' ','AAPL') + apikey
output = json.load(urlopen(uu))
norm = pd.read_json(uu)

In [ ]:
pd.read_json(json.dumps(pd.read_json(uu).iloc[0,1]))

In [ ]:
ttdf = pd.read_json(uu)
print(ttdf)
            
if ttdf.shape == (1,2):
    print(ttdf)
    ttdf = pd.read_json(json.dumps(pd.read_json(uu).iloc[0,1]))
    print(ttdf)

In [43]:
norm['historical'] = norm['historical'].apply(lambda x: pd.Series(x)) 

,date,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime
0,2021-02-12,134.350006,135.529999,133.690002,135.369995,135.369995,60029300.0,60029300.0,1.01999,0.759,134.86333,"February 12, 21",0.00759
1,2021-02-11,135.899994,136.389999,133.770004,135.130005,135.130005,64280000.0,64280000.0,-0.76999,-0.567,135.09667,"February 11, 21",-0.00567
2,2021-02-10,136.479996,136.990005,134.399994,135.389999,135.389999,73046600.0,73046600.0,-1.09000,-0.799,135.59333,"February 10, 21",-0.00799
3,2021-02-09,136.619995,137.880005,135.850006,136.009995,136.009995,76774200.0,76774200.0,-0.61000,-0.446,136.58000,"February 09, 21",-0.00446
4,2021-02-08,136.029999,136.960007,134.919998,136.910004,136.910004,71297200.0,71297200.0,0.88000,0.647,136.26334,"February 08, 21",0.00647
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10123,1980-12-18,0.118862,0.119420,0.118862,0.118862,0.093615,73449600.0,73449600.0,0.00000,0.000,0.11905,"December 18, 80",0.00000
10124,1980-12-17,0.115513,0.116071,0.115513,0.115513,0.090978,86441600.0,86441600.0,0.00000,0.000,0.11570,"December 17, 80",0.00000
10125,1980-12-16,0.113281,0.113281,0.112723,0.112723,0.088780,105728000.0,105728000.0,-0.00056,-0.494,0.11291,"December 16, 80",-0.00494
10126,1980-12-15,0.122210,0.122210,0.121652,0.121652,0.095813,175884800.0,175884800.0,-0.00056,-0.458,0.12184,"December 15, 80",-0.00458
